<a href="https://colab.research.google.com/github/aminmarani/Topic-Labeling/blob/master/topic2html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#installing packages necessary for Rmallet

In [9]:
install.packages(c('dplyr','rJava','mallet','reshape2','reticulate'))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘png’




In [11]:
#Import necessary libraries
library(dplyr)
library (rJava)
.jinit(parameters="-Xmx12g") #Give rJava enough memory
library(mallet)
library(ggplot2)
library(reshape2)
library(jsonlite)
library(reticulate)
source("topic_modeler.R")

[1] 0

#reading data from text file

In [15]:
#Import text to a data variable (requires formatting data once imported)
if(exists("data", mode = "list")){
  rm(data) #Necessary due to different sizes of data
}
text = scan("ap.txt", what="", encoding="UTF-8", sep="\n")
text = text[!startsWith(text, "<")]
data <- data.frame(as.character(1:length(text)), stringsAsFactors = FALSE)
names(data) <- "id"
data$body <- text
#Use rbind to create a documents variable from the data.frame
documents <- rbind(data.frame(title=data$id, text=data$body, stringsAsFactors=F))
docs.and.topics <- cbind(data, createModel(documents, stopwords="ap/mallet_en_stopwords.txt", 
                                           label_method="topn", label_len=20,
                                           model_use_public=F, n.topics=100)) #Make sure public.model is correct if model_use_public = T

[1] "Building model: Method: lda"
[1] "Building mallet instance ..."
[1] "Building topic trainer ..."
[1] "Loading documents in mallet instance into topic trainer ..."
[1] "Training model ..."
[1] "Training complete."
[1] "Creating labels: Method: topn"
[1] "Building final output ..."
[1] "Done!"


#injecting topics and top-documents into html file named "test.html"

In [16]:
s = "topic_id"
for (i in 0:19)
  s = paste(s ,",term" , i )
a = colnames(docs.and.topics)
a[4] = s
write.csv(a[4:54],'AP2.csv',sep=",")
#save data into a html file
txt = ''
topic.names = colnames(docs.and.topics)
for (i in 5:54)
{
  #split string to get top3 words for header
  topic.names.split = strsplit(topic.names[i],' ')
  topic.topn = ''
  for (j in 1:length(topic.names.split[[1]]))
  {
    topic.topn = paste(topic.topn,topic.names.split[[1]][j])
  }
  #make a toggle division with 3 top words of the topic
  txt = paste(txt,'\n<details> \n \t
                <summary>',topic.topn,
              '</summary>')
  
  #put top 5 document with most of that topic
  topics.scores = docs.and.topics[,i]
  sorted.scores = sort(topics.scores,index.return=TRUE,decreasing = TRUE)
  docs.topn = docs.and.topics[sorted.scores$ix[1:5],2]
  #add top n-doc with bold topn words
  txt = paste(txt,'\n <p>') #start the tag
  #write each doc to the html file
  for (j in 1:5)
  {
    txt = paste(txt,docs.and.topics[sorted.scores$ix[j],i], ' \n ')
    txt.split = strsplit(docs.topn[j],' ') #split the text to search for topn terms
    #check if a term is topn or not
    term.bold = integer(length(txt.split[[1]]))
    for (k in 1:length(txt.split[[1]]))#check every single term in doc
    {
      if(txt.split[[1]][k] %in% topic.names.split[[1]])
        term.bold[k] = 1
      else
        term.bold[k] = 0
    }
    for (k in 1:length(txt.split[[1]]))#check every single term in doc
    {
      if(term.bold[k]) #add bold
        txt = paste(txt,'<b>',txt.split[[1]][k],'</b>')
      else #add text
        txt = paste(txt,txt.split[[1]][k])
    }
    txt = paste(txt, '</p>') #finish the tag
  }
  txt = paste(txt, '</details>') #finish the details tag
  #<p>Here you can put some content...</p>
}
write.table(txt, file = 'test.html')

Warning message in write.csv(a[4:54], "AP2.csv", sep = ","):
“attempt to set 'sep' ignored”


#Showing the results


In [17]:
htmltools::includeHTML('test.html')

"x"
"1" " 
<details> 
 	
                <summary>  eastern pilots union airline northwest airlines labor air united employees orion service faa ual teamsters eastern's flight unions grumman machinists </summary> 
 <p> 0.449753046444687  
   Union <b> pilots </b> who struck Eastern Airlines are entitled to replace trainees hired to help the carrier rebuild, a federal judge said. Thursday's ruling by U.S. District Judge Edward Davis apparently affects only those who were in training when time the <b> pilots </b> ended their walkout, said E.J. Breen, spokesman for the Air Line Pilots Association. Still, Breen said, ``It's clearly a major victory.'' All but 200 of the 3,600 <b> union </b> <b> pilots </b> walked off the job in sympathy with Machinists who struck the Miami-based <b> airline </b> in March 1989. Pilots ended their sympathy walkout in November, but fewer than 50 have been called back to Eastern, <b> pilots </b> association officials say. The Machinists are still striking. At least 404 <b> pilots </b> were in training when the <b> pilots </b> returned to work, the <b> pilots </b> association says. The <b> airline </b> estimates only 180 jobs are affected by the ruling, Eastern spokeswoman Karen Ceremsak said. Further hearings likely will be determine the exact number of affected jobs, Breen said. Eastern said in a statement it would appeal the decision. The <b> airline </b> predicted the ruling wouldn't immediately affect current <b> employees </b> or operations. Many <b> pilots </b> took jobs with other <b> airlines </b> after the strike began March 4, 1989, but about 1,400 former Eastern <b> pilots </b> may still be interested in returning, pilots' association spokesman Lou Baldwin said. Davis' ruling stems from the association's challenge of Eastern's claim that the formerly striking <b> pilots </b> were not needed. Davis agreed that the federal Railway Labor Act guarantees returning strikers job rights ahead of newly hired trainees. Davis said Eastern violated the Railway Labor Act by hiring pilot trainees not already qualified to fly regular flights. The act was devised to provide ``uninterrupted transportation service,'' the ruling said. Eastern sought bankruptcy protection five days after the strike began. It presently operates about 800 flights, down from its pre-strike <b> service </b> of 1,040. Ms. Ceremsak said she does not believe Davis' ruling must receive bankruptcy court approval. If the ruling is upheld on appeal, all returning <b> pilots </b> would need to go through several weeks of recertification training under the Federal Aviation Administration, Eastern officials said. Eastern currently has about 1,900 pilots, including those trained before the strike was settled, <b> pilots </b> hired from other <b> airlines </b> and <b> union </b> <b> pilots </b> who later crossed picket lines. Baldwin said the Eastern <b> pilots </b> trainees' annual pay is about $27,000. The average pre-strike pay at Eastern for fully accredited <b> pilots </b> was about $65,000, he said. </p> 0.429109398698947  
   The chairman of Texas Air Corp. Friday defended a decision by the company's subsidiary, Eastern Airlines, to lay off 4,000 <b> employees </b> and cut <b> service </b> as a way to ``stabilize the company'' by cutting losses. Frank Lorenzo, testifying at a federal court hearing on Eastern's plans to eliminate <b> service </b> to 14 cities, said the layoffs and schedule changes were a necessary step to save the financially troubled carrier. ``If Eastern's management is not permitted to take the actions to save this company, then Eastern may run out of cash,'' Lorenzo said, rejecting suggestions by <b> union </b> attorneys that he was trying to bust organized <b> labor </b> at Eastern by transferring the carrier's assets to Continental Airlines, another Texas Air property. Three <b> labor </b> <b> unions </b> that represent Eastern <b> employees </b> have challenged the plans and obtained a temporary restraining order 